In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# We Model Directly

In [2]:
df = pd.read_csv('StudentPerformanceFactors.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6607 entries, 0 to 6606
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Hours_Studied               6607 non-null   int64 
 1   Attendance                  6607 non-null   int64 
 2   Parental_Involvement        6607 non-null   object
 3   Access_to_Resources         6607 non-null   object
 4   Extracurricular_Activities  6607 non-null   object
 5   Sleep_Hours                 6607 non-null   int64 
 6   Previous_Scores             6607 non-null   int64 
 7   Motivation_Level            6607 non-null   object
 8   Internet_Access             6607 non-null   object
 9   Tutoring_Sessions           6607 non-null   int64 
 10  Family_Income               6607 non-null   object
 11  Teacher_Quality             6529 non-null   object
 12  School_Type                 6607 non-null   object
 13  Peer_Influence              6607 non-null   obje

In [ ]:
df.drop(inplace = True, labels = ['Parental_Involvement', 'Internet_Access', 'Family_Income', 'Parental_Education_Level', 'Gender', 'Tutoring_Sessions'], axis = 1)
df.dropna(inplace = True)

In [5]:
df.columns

Index(['Hours_Studied', 'Attendance', 'Access_to_Resources',
       'Extracurricular_Activities', 'Sleep_Hours', 'Previous_Scores',
       'Motivation_Level', 'Tutoring_Sessions', 'Teacher_Quality',
       'School_Type', 'Peer_Influence', 'Physical_Activity',
       'Learning_Disabilities', 'Distance_from_Home', 'Exam_Score'],
      dtype='object')

## Modeling, CatBoost

In [8]:
categorical = df.select_dtypes(include = 'object')
categorical

,Access_to_Resources,Extracurricular_Activities,Motivation_Level,Teacher_Quality,School_Type,Peer_Influence,Learning_Disabilities,Distance_from_Home
0,High,No,Low,Medium,Public,Positive,No,Near
1,Medium,No,Low,Medium,Public,Negative,No,Moderate
2,Medium,Yes,Medium,Medium,Public,Neutral,No,Near
3,Medium,Yes,Medium,Medium,Public,Negative,No,Moderate
4,Medium,Yes,Medium,High,Public,Neutral,No,Near
...,...,...,...,...,...,...,...,...
6602,Medium,No,Medium,Medium,Public,Positive,No,Near
6603,Medium,No,Medium,High,Public,Positive,No,Near
6604,Low,Yes,Low,Medium,Public,Negative,No,Near
6605,High,Yes,High,Medium,Private,Positive,No,Far


In [9]:
for col in categorical.columns:
    df[col] = categorical[col].map(lambda x: x.lower())

df.head()

,Hours_Studied,Attendance,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Tutoring_Sessions,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Distance_from_Home,Exam_Score
0,23,84,high,no,7,73,low,0,medium,public,positive,3,no,near,67
1,19,64,medium,no,8,59,low,2,medium,public,negative,4,no,moderate,61
2,24,98,medium,yes,7,91,medium,2,medium,public,neutral,4,no,near,74
3,29,89,medium,yes,8,98,medium,1,medium,public,negative,4,no,moderate,71
4,19,92,medium,yes,6,65,medium,3,high,public,neutral,4,no,near,70


In [16]:
from sklearn.model_selection import train_test_split

x, y = df[df.columns[:-1]], df['Exam_Score']

x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle = True, random_state = 420)

In [21]:
from imblearn.over_sampling import RandomOverSampler

sampler = RandomOverSampler()
x_train, y_train = sampler.fit_resample(x_train, y_train)

In [6]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier()

cat.fit(x_train, y_train, cat_features = categorical.columns.tolist())

NameError: name 'x_train' is not defined

In [19]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [20]:
y_hat = cat.predict(x_test)

mse = mean_squared_error(y_test, y_hat)
mae = mean_absolute_error(y_test, y_hat)
r2 = r2_score(y_test, y_hat)

print("R2: ", r2)
print("MSE: ", mse)
print("MAE: ", mae)

R2:  0.5923875212259497
MSE:  6.458539603960396
MAE:  1.254331683168317


# Save the model

In [26]:
cat.save_model('catboost_ml_model', format = 'cbm')